In [19]:
from decouple import config
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from operator import itemgetter

from langchain.chains import create_sql_query_chain, LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain.evaluation import load_evaluator, EmbeddingDistance
from langchain_community.embeddings import HuggingFaceEmbeddings

import pandas as pd
import psycopg2
import json

In [6]:
OPENAI_API_KEY = config("OPENAI_API_KEY")
DB_USER = config('DB_USER')
DB_PASSWORD = config('DB_PASSWORD')
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = "food_security"
SAMPLE_QUESTIONS = {"low-birthweight": "Which region has the highest number of children born with low birth weights?",
                    "vaccine_rates": "Which vaccine has the lowest vaccination percentage?",
                    "vaccine_rates_all": "What percentage of children received all vaccines before 12 months",
                    "kids_in_sch": "Whats average percentage of children who are in preschool",
                    "vaccines": "which vaccines did children get in Tunisia?"
                    }
                    
# Create the database URL
DATABASE_URL = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
FILE_SQL_EXAMPLES_EN = "sql_examples_en.json"
USE_BEST_MATCHING_COLUMNS = False

## Prompt Templates

### Create Prompt to Select best Table

In [7]:
def connect_to_database(database_url=DATABASE_URL):
    """Connects to a postgreSQL


    Parameters
    ----------
    database_url : String
        postgreSQL database connection URL, by default DATABASE_URL
    """
    # conn = psycopg2.connect(f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD}")
    conn = psycopg2.connect(database_url)

    cur = conn.cursor()

    # Query to get table names and column names
    cur.execute("SELECT table_name, description FROM table_metadata")
    tables = cur.fetchall()

    cur.execute("SELECT table_name, column_name, description FROM column_metadata")
    columns = cur.fetchall()

    cur.close()
    conn.close()

    return tables, columns

In [8]:
def find_best_table_prompt(user_query, tables, columns, 
                           return_chain=True, llm=None):# Define the template for selecting the best table
    template = """
    You are a database assistant. Given the following tables and columns with their descriptions, select the best table that matches the user's query.

    Tables and Columns:
    {table_info}

    User Query:
    {user_query}

    Provide the output in the following JSON format:
    {{
        "best_matching_table": {{
            "table_name": "<best_table_name>",
            "description": "<best_table_description>"
        }}
    }}
    """
    # Prepare the table_info string including descriptions for each table and its columns
    table_info = ""
    for table in tables:
        table_name, table_description = table
        table_info += f"Table: {table_name} - {table_description}\n"
        table_columns = [col for col in columns if col[0] == table_name]
        for column in table_columns:
            _, column_name, column_description = column
            table_info += f"    Column: {column_name} - {column_description}\n"
        table_info += "\n"

    # Create the PromptTemplate
    prompt_template = PromptTemplate(
        template=template,
        input_variables=["table_info", "user_query"]
    )

    # Format the template 
    formatted_prompt = prompt_template.format(table_info=table_info, user_query=user_query)

    if return_chain:
        # Create the chain using the ChatOpenAI model and the PromptTemplate
        chain = LLMChain(llm=llm,prompt=prompt_template)
        return chain, {"table_info": table_info, "user_query": user_query}

    return formatted_prompt


In [9]:
def get_columns_info(table_name, columns):
    columns_info = ""
    for column in columns:
        table, column_name, column_description = column
        if table == table_name:
            columns_info += f"    Column: {column_name} - {column_description}\n"
    return columns_info


In [10]:
def find_best_columns_prompt(user_query, best_matching_table, columns, 
                       return_chain=True, llm=None):
    # Define the template for selecting the relevant columns
    column_template = """
    You are a database assistant. Given the following columns for the table '{table_name}', select the columns that are most relevant to the user's query.

    Table Description: {table_description}

    Columns:
    {columns_info}

    User Query:
    {user_query}

    Relevant Columns:
    """

    columns_info = get_columns_info(best_matching_table["table_name"], columns)

    # Create the PromptTemplate for column selection
    column_prompt_template = PromptTemplate(
        template=column_template,
        input_variables=["table_name", "table_description", "columns_info", "user_query"]
    )

    # Example usage of the template with a user query
    formatted_column_prompt = column_prompt_template.format(
        table_name=best_matching_table["table_name"],
        table_description=best_matching_table["description"],
        columns_info=columns_info,
        user_query=user_query
    )

    # Prepare the context for running the chain
    context = {
        "table_name": best_matching_table["table_name"],
        "table_description": best_matching_table["description"],
        "columns_info": columns_info,
        "user_query": user_query}

    if return_chain:
        chain = LLMChain(llm=llm,prompt=column_prompt_template)
        return chain, context

    return formatted_column_prompt

In [11]:
def load_sql_examples(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [12]:
def create_sql_prompt(examples, best_matching_table, columns_metadata, 
                      use_best_matching_columns=False, llm=None):
    """
    Creates a FewShotPromptTemplate for generating SQL queries based on table and column metadata.

    This function generates a prompt template that includes detailed information about the table and its columns.
    The generated prompt instructs a language model (LLM) to create a syntactically correct SQL query based on
    user input. If the table contains a date column and the user does not specify a date, the prompt also instructs
    the LLM to retrieve the most recent data available.

    Parameters
    ----------
    examples : list of dict
        A list of example inputs and corresponding SQL queries. Each example should be a dictionary with 'input' and 'query' keys.
    best_matching_table : dict
        A dictionary containing the best matching table information with 'table_name' and 'description' keys.
    columns_metadata : list of tuples
        A list of tuples containing columns metadata. Each tuple should include 'table_name', 'column_name', and 'description'.
    use_best_matching_columns : bool, optional
        A flag indicating whether to use only the best-matching columns (if True) or all columns in the table (if False). Default is True.

    Returns
    -------
    sql_prompt : FewShotPromptTemplate
        A FewShotPromptTemplate object that can be used with an LLM to generate SQL queries.
    """
    # Prepare table_info string based on the best matching table and columns
    # table_info = f"Table: {best_matching_table['table_name']} - {best_matching_table['description']}\n"
    columns_info = "Columns:\n"
    has_date_column = False

    # Determine which columns to use: best-matching or all columns
    if use_best_matching_columns:
        # If using best_matching_columns, use those provided (filtering columns_metadata based on matching logic)
        columns_to_use = columns_metadata  # Assuming columns_metadata is already filtered
    else:
        # Use all columns for the given table from columns_metadata
        columns_to_use = [col for col in columns_metadata if col[0] == best_matching_table['table_name']]

    # Construct the columns_info string
    for column in columns_to_use:
        table_name, column_name, column_description = column
        columns_info += f"    Column: {column_name} - {column_description}\n"
        if 'date' in column_name.lower():
            has_date_column = True

    # Create FewShot Prompt with instructions for handling most recent data
    example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")

    # Add a special instruction if the table has a date column
    recent_data_instruction = (
        "If the user does not specify a date, retrieve the most recent data available by ordering the results "
        "by the date column in descending order."
    ) if has_date_column else ""

    # Combine table_info and columns_info in the prompt
    sql_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix=(
            "You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query to run. "
            "Unless otherwise specified, do not return more than {top_k} rows.\n\n"
            "Here is the relevant table information:\n{table_info}\n\n"
            f"{recent_data_instruction}\n\n"
            "Below are a number of examples of questions and their corresponding SQL queries."
        ),
        suffix="User input: {input}\nSQL query: ",
        input_variables=["input", "table_info", "top_k"],
    )
    
    
    return sql_prompt

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import FewShotPromptTemplate

def create_custom_sql_query_chain(llm, db, examples, best_matching_table, columns_metadata, use_best_matching_columns=False):
    """
    Custom function to create an SQL query chain using the provided LLM and SQL prompt.

    Parameters
    ----------
    llm : Any
        The language model (e.g., ChatOpenAI) to be used for generating the SQL query.
    db : SQLDatabase
        The database object for executing the generated SQL queries.
    examples : list of dict
        A list of example inputs and corresponding SQL queries.
    best_matching_table : dict
        A dictionary containing the best matching table information with 'table_name' and 'description' keys.
    columns_metadata : list of tuples
        A list of tuples containing columns metadata.
    use_best_matching_columns : bool, optional
        A flag indicating whether to use only the best-matching columns or all columns in the table. Default is False.

    Returns
    -------
    write_query : LLMChain
        An LLMChain object ready to generate and execute SQL queries based on user input.
    """
    # Generate the sql_prompt using the provided function
    sql_prompt = create_sql_prompt(
        examples=examples, 
        best_matching_table=best_matching_table, 
        columns_metadata=columns_metadata,
        use_best_matching_columns=use_best_matching_columns
    )

    # Create the LLMChain manually
    return LLMChain(llm=llm, prompt=sql_prompt)

# Initialize LLM and other components
engine = create_engine(DATABASE_URL)
db = SQLDatabase(engine=engine, ignore_tables=['table_metadata', 'column_metadata'])

# Use the custom wrapper to create the chain
write_query = create_custom_sql_query_chain(
    llm=llm,
    db=db,
    examples=examples,
    best_matching_table=best_table_output,
    columns_metadata=best_columns_output if USE_BEST_MATCHING_COLUMNS else columns,
    use_best_matching_columns=USE_BEST_MATCHING_COLUMNS
)

# Create the answer chain
answer_chain = create_answer_chain(llm)

# Put everything together
master_chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_chain
)

# Invoke the master chain with the required inputs
response = master_chain.invoke({
    "input": user_question,  # This is your user query
    "top_k": 10              # You can adjust this value as needed for the number of rows to return
})

print(response)


In [58]:
def create_answer_chain(llm):
    # Define the prompt template with emphasis on including units, time-specific details, and using the latest data when time is not specified
    answer_prompt = PromptTemplate.from_template(
        """
        You are a knowledgeable assistant. Given the following user question and SQL result, answer the question accurately.
        
        Always ensure to:
        1. Include appropriate units in your answer (e.g., Kwacha per kg, liters, etc.).
        2. Specify the time period or date if the question implies or explicitly asks for it.
        3. If the user does not specify a time, provide the most recent information available in the database and clearly state that this is the latest data.

        For example, if the user asks "What's the price of Maize?", your answer should include the price with the correct unit and mention that this is the most recent price, e.g., "The most recent price of Maize is 60 Kwacha per kg."
        If the user asks about a specific time period, such as "What's the price of Maize for May 2024?", include the time in your answer, e.g., "The price of Maize in May 2024 is 60 Kwacha per kg."

        Question: {question}
        SQL Result: {result}
        Answer: """
    )

    return answer_prompt | llm | StrOutputParser()

In [52]:
def run_sql_chain(user_query, best_table_info, columns_info, best_columns=None, llm=None):
    """
    Executes an SQL query generation chain using a language model (LLM) based on the user query, 
    best matching table, and columns information.

    This function loads example SQL queries, creates an SQL prompt tailored to the best matching 
    table and its columns, and then executes a chain that generates and executes an SQL query. 
    The response is returned after processing the generated query.

    Parameters
    ----------
    user_query : str
        The user's query for which an SQL query needs to be generated.
    best_table_info : dict
        A dictionary containing the best matching table information with 'table_name' and 'description' keys.
    columns_info : list of tuples
        A list of tuples containing columns metadata for the table. Each tuple includes 'table_name', 
        'column_name', and 'description'.
    best_columns : list of tuples, optional
        A list of tuples containing the best matching columns metadata, if available. If provided, 
        the SQL prompt will be generated using only these columns. Default is None.
    llm : Any, optional
        The language model (e.g., ChatOpenAI) to be used for generating the SQL query. Default is None.

    Returns
    -------
    response : Any
        The response from the executed SQL query chain, typically containing the results of the SQL query.

    Example
    -------
    response = run_sql_chain(
        user_query="What is the price of maize?",
        best_table_info={"table_name": "maize_prices", "description": "Contains maize price data"},
        columns_info=[("maize_prices", "price", "Price of maize"), ("maize_prices", "date", "Date of the price entry")],
        llm=ChatOpenAI()
    )
    print(response)
    """
    # Load examples and create prompts
    examples = load_sql_examples(file_path=FILE_SQL_EXAMPLES_EN)
    
    # Create SQL Query
    if USE_BEST_MATCHING_COLUMNS and best_columns:
        sql_prompt = create_sql_prompt(
            examples=examples, 
            best_matching_table=best_table_info, 
            columns_metadata=best_columns, 
            use_best_matching_columns=True
        )
    else:
        sql_prompt = create_sql_prompt(
            examples=examples, 
            best_matching_table=best_table_info, 
            columns_metadata=columns_info
        )

    # Initialize LLM and other components
    best_table = best_table_info['table_name']
    engine = create_engine(DATABASE_URL)
    db = SQLDatabase(engine=engine, ignore_tables=['table_metadata', 'column_metadata'])

    execute_query = QuerySQLDataBaseTool(db=db)
    write_query = create_sql_query_chain(llm, db, sql_prompt)

    # Create the answer chain
    answer_chain = create_answer_chain(llm)

    # Put everything together
    master_chain = (
        RunnablePassthrough.assign(query=write_query).assign(
            result=itemgetter("query") | execute_query
        )
        | answer_chain
    )

    # Invoke the master chain and return the response
    response = master_chain.invoke({
        "question": user_query, 
        "top_k": 3,
        "table_info": best_table
    })
    return response

In [66]:
def process_sql_query(user_question, use_huggingface=False):
    """
    Processes a user's question by generating and executing an SQL query using a language model (LLM). 
    Optionally, uses a Hugging Face model or defaults to OpenAI's GPT-3.5-turbo.

    This function first initializes the appropriate LLM based on the `use_huggingface` flag. It then 
    retrieves metadata information, identifies the best matching table and relevant columns, and 
    executes the SQL query based on the processed information.

    Parameters
    ----------
    user_question : str
        The user's question for which an SQL query needs to be generated and executed.
    use_huggingface : bool, optional
        A flag to determine whether to use a Hugging Face model instead of the default OpenAI model. 
        Default is False.

    Returns
    -------
    output : Any
        The output from the executed SQL query chain, typically containing the results of the SQL query.

    Example
    -------
    output = process_sql_query(
        user_question="What is the price of maize?",
        use_huggingface=False
    )
    print(output)
    """
    # Initialize LLM
    # To Do: add Hugging Face LLM
    if use_huggingface:
        pass  # Hugging Face LLM initialization can be added here
    else:
        openai_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)
   
    # Retrieve the metadata info (tables and columns)
    tables, columns = connect_to_database()

    # Chain 1: Find the Best Table
    best_table_chain, context = find_best_table_prompt(user_question, tables, columns, llm=openai_llm)
    best_table_output_str = best_table_chain.run(**context)

    # Convert the string output to a dictionary
    try:
        best_table_output = json.loads(best_table_output_str)['best_matching_table']
    except json.JSONDecodeError:
        print("Error: The output is not valid JSON.")
        best_table_output = None

    # Chain 2: Find Relevant Columns
    best_columns_chain, context = find_best_columns_prompt(user_question, best_table_output, columns, llm=openai_llm)
    best_columns_output = best_columns_chain.run(**context)

    # Retrieve result 
    output = run_sql_chain(
        user_query=user_question, 
        best_table_info=best_table_output, 
        columns_info=columns, 
        best_columns=best_columns_output, 
        llm=openai_llm
    )
    
    return output


In [54]:
question = "Whats the price of Maize in Rumphi?"
response = process_sql_query(user_question=question)
print(response)

The most recent price of Maize in Rumphi is 764.45 Kwacha.


In [49]:
response['text']

'The most recent price of Maize in Rumphi is 764.445 Kwacha.'

## Evaluate results 

In [30]:
def evaluate_with_embedding_distance(df, lan='English', 
                                     distance='cosine', ref_response_col='response-1'):


    # Filter the dataframe to keep only instances for that language
    df = df.query('language == @lan')
    
    # Add columns to keep LLM-response and eval score
    df['llm_response'] = None
    score_name = f"score_{distance}"
    df[score_name] = None

    
    # Setup evaluator
    if distance == "cosine":
        dist_metric = EmbeddingDistance.COSINE
    elif distance == "euclidean":
        dist_metric = EmbeddingDistance.EUCLIDEAN

    embedding_model = HuggingFaceEmbeddings()
    hf_evaluator = load_evaluator("embedding_distance", distance_metric=dist_metric, 
                              embeddings=embedding_model)
    for idx, row in df.iterrows():
        question = row['question']
        reference_response = row[ref_response_col]
        llm_response = process_sql_query(question)
        print('LLM-Response', "===>", llm_response)
        if not llm_response:
            print('Blank response')
        score = hf_evaluator.evaluate_strings(prediction=llm_response, 
                                              reference=reference_response)
        df.loc[idx, 'llm_response'] = llm_response
        df.loc[idx, score_name] = score['score']
    
    return df

In [59]:
engine = create_engine(DATABASE_URL)
db = SQLDatabase(engine=engine, ignore_tables=['table_metadata', 'column_metadata'])

In [63]:
table_info = db.table_info
len(table_info)

1741

In [68]:
df_eval = pd.read_csv("data/raw/eval-set.csv")
df_res = evaluate_with_embedding_distance(df=df_eval, lan='English', distance='cosine')

/var/folders/k5/p4nvl2pj4gq2ks0j7qvrhwbh0000gp/T/ipykernel_66207/747459483.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llm_response'] = None
/var/folders/k5/p4nvl2pj4gq2ks0j7qvrhwbh0000gp/T/ipykernel_66207/747459483.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[score_name] = None
/Users/dunstanmatekenya/Library/CloudStorage/GoogleDrive-dmatekenya@gmail.com/My Drive/phosoAI-whatsapp-app-aws/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `

LLM-Response ===> The price of Maize in Rumphi is 764.45 Kwacha. This is the most recent price available in the database.
LLM-Response ===> The price of rice in Lilongwe district is 1867.50 Kwacha per kg. This is the most recent price available in the database.
LLM-Response ===> The district with the lowest price for groundnuts is Karonga, with a price of 1256.48 Kwacha.
LLM-Response ===> Based on the SQL result, the locations where you can get cheap beans are Chikhwawa, Malomo, and Nchalo, where the price is 1200 Kwacha per kg. In comparison, Ngabu offers beans at a slightly higher price of 1300 Kwacha per kg, and Nkhata Bay has beans priced at 1336.486 Kwacha per kg.
LLM-Response ===> Based on the SQL result, the cheapest place to buy Maize is Karonga with a price of 473.60 Kwacha per kg.
LLM-Response ===> The best place to sell soya, based on the available data, is Mtakataka.
LLM-Response ===> The most recent price of beans is 2704.91 Kwacha per kg.
LLM-Response ===> The district th

In [32]:
df_res

,question,language,response-1,response-2,llm_response,score_cosine
13,How much is maize in Dowa?,English,"Based on the most recent prices of May, 2024, ...",NaN,The price of maize in Dowa is 562.5 Kwacha per...,0.07533
11,Where can I buy soya?,English,You should look for soy beans in the following...,NaN,"Based on the available data, you can buy soya ...",0.471374
5,Where is the best place to sale soya?,English,"Based on the most recent prices of May 2024, s...",NaN,"The best place to sell soya, based on the avai...",0.345378
17,current price of Maize in the country?,English,"Based on the most recent prices of May 2024, t...",NaN,The most recent price of Maize in the country ...,0.067668


In [ ]:
question = "Whats the price of Maize?"
response = process_sql_query(user_question=question, llm=openai_chat_model)


create_sql_prompt(examples, best_matching_table, columns_metadata)

In [ ]:
response